# Purpose
The purpose of this notebook is to calculate the SMS chiral potential up to $j = 20$ and save them in a h5 file. Here, we append $k_0$ to the mesh and calculate the potentials at this point for the LEC-dependent partial waves. Interpolation is done for the non-LEC-dependent partial waves.

# Notebook Setup

## Import Python libraries

In [1]:
import sys
import h5py
import numpy as np
from ruamel.yaml import YAML

%load_ext autoreload
%autoreload 2

sys.path.insert(0, '../../')

## Import local libraries

In [2]:
from emulate_kvp import BuildMesh
from emulate_kvp import SMSChiralPotential
from emulate_kvp import uncoupled_potential, coupled_potential
from emulate_kvp.kinematics import e_lab_to_k_cm, avoid_pv_sing
from emulate_kvp import mass_proton, mass_neutron
from emulate_kvp.potential_functions import GetFullPotential

# Calculate potentials

## Parameters

In [3]:
### MESH INPUT PARAMETERS ###
ki = 0
cut_reg1 = 3 # Mesh cutoff of first region
cut_reg2 = 6 # Mesh cutoff of second region
kf = 8

pts_reg1 = 40 # Total points in first region
pts_reg2 = 20 # Total points in second region
pts_reg3 = 20 # Total points in third region
Ntot = pts_reg1 + pts_reg2 + pts_reg3

### POTENTIAL INPUT PARAMETERS ###
interaction = 'np' # Choice of interaction
chiral_order = 5 # Order of EFT
cutoff = 4 # Choice of cutoff
jmax = 20 # Momentum

## Build mesh

In [4]:
mesh_nodes = [ki, cut_reg1, cut_reg2, kf]
pts_per_region = [pts_reg1, pts_reg2, pts_reg3]
mesh = BuildMesh(mesh_nodes, pts_per_region)
nodes, weights = mesh.nodes, mesh.weights

E_MeV = np.linspace(1, 350, num=350)
degrees = np.arange(0.1, 180)

E_MeV = avoid_pv_sing(E_MeV, nodes, mass_neutron, mass_proton, True)[0]
E_MeV[0] = 0.1
k_cm = e_lab_to_k_cm(E_MeV, mass_neutron, mass_proton, relativistic=True)[1]

## Get coupling constants

In [5]:
if (cutoff == 1):
    value = 400
elif (cutoff == 2):
    value = 450
elif (cutoff == 3):
    value = 500
elif (cutoff == 4):
    value = 550
    
location = '../../../data/' + str(value) + 'MeV/'

yaml = YAML(typ="safe")
with open(location + 'sms_lecs_n4lop_lam' + str(value) + '.yaml', "r") as input_file:
    lecs_yaml = yaml.load(input_file)

In [6]:
lecs_1S0 = [val for key, val in lecs_yaml.items() if "1S0" in key]
lecs_3P0 = [val for key, val in lecs_yaml.items() if "3P0" in key]
lecs_1P1 = [val for key, val in lecs_yaml.items() if "1P1" in key]
lecs_3P1 = [val for key, val in lecs_yaml.items() if "3P1" in key]
lecs_1D2 = [val for key, val in lecs_yaml.items() if "1D2" in key]
lecs_3D2 = [val for key, val in lecs_yaml.items() if "3D2" in key]
lecs_1F3 = [val for key, val in lecs_yaml.items() if "1F3" in key]
lecs_3F3 = [val for key, val in lecs_yaml.items() if "3F3" in key]

lecs_3S1_3D1 = [val for key, val in lecs_yaml.items() if "3S1" in key or "3D1" in key]
lecs_3P2_3F2 = [val for key, val in lecs_yaml.items() if "3P2" in key or "3F2" in key]
lecs_3D3_3G3 = [val for key, val in lecs_yaml.items() if "3D3" in key or "3G3" in key]
lecs_3F4_3H4 = [val for key, val in lecs_yaml.items() if "3F4" in key or "3H4" in key]

## Calculate potential

In [7]:
%%time
my_sms = SMSChiralPotential(chiral_order, interaction, cutoff)
spectral, contacts = my_sms.get_chiral_potential(nodes, jmax)

Semilocal momentum-space chiral NN potential at N4LO [Q^5] + N5LO [Q^6]
Contacts in 3F2, 1F3, 3F3, 3F4
Cutoff value: lambda = 550 MeV


jmom = 0
jmom = 1
jmom = 2
jmom = 3
jmom = 4
jmom = 5
jmom = 6
jmom = 7
jmom = 8
jmom = 9
jmom = 10
jmom = 11
jmom = 12
jmom = 13
jmom = 14
jmom = 15
jmom = 16
jmom = 17
jmom = 18
jmom = 19
jmom = 20
CPU times: user 13min 41s, sys: 7.8 s, total: 13min 49s
Wall time: 14min 6s


In [8]:
%%time
full_pot = GetFullPotential(k_cm, nodes, jmax, my_sms)

E: 25
E: 50
E: 75
E: 100
E: 125
E: 150
E: 175
E: 200
E: 225
E: 250
E: 275
E: 300
E: 325
E: 350
CPU times: user 1h 11min 35s, sys: 24.3 s, total: 1h 12min
Wall time: 1h 11min 47s


## Uncoupled partial waves

### Partial wave: 1S0

In [9]:
jmom, wave = 0, '1S0'
pot_1S0, spec_1S0, cont_1S0 = uncoupled_potential(jmom, lecs_1S0, 
                                                  spectral, contacts, wave)
V_1S0, V0_1S0, V1_1S0 = full_pot.no_interp_pot(jmom, wave, lecs_1S0, 
                                               pot_1S0, spec_1S0, cont_1S0)

### Partial wave: 3P0

In [10]:
jmom, wave = 0, '3P0'
pot_3P0, spec_3P0, cont_3P0 = uncoupled_potential(jmom, lecs_3P0, 
                                                  spectral, contacts, wave)
V_3P0, V0_3P0, V1_3P0 = full_pot.no_interp_pot(jmom, wave, lecs_3P0, 
                                               pot_3P0, spec_3P0, cont_3P0)

### Partial wave: 1P1

In [11]:
jmom, wave = 1, '1P1'
pot_1P1, spec_1P1, cont_1P1 = uncoupled_potential(jmom, lecs_1P1, 
                                                  spectral, contacts, wave)
V_1P1, V0_1P1, V1_1P1 = full_pot.no_interp_pot(jmom, wave, lecs_1P1, 
                                               pot_1P1, spec_1P1, cont_1P1)

### Partial wave: 3P1

In [12]:
jmom, wave = 1, '3P1'
pot_3P1, spec_3P1, cont_3P1 = uncoupled_potential(jmom, lecs_3P1, 
                                                  spectral, contacts, wave)
V_3P1, V0_3P1, V1_3P1 = full_pot.no_interp_pot(jmom, wave, lecs_3P1, 
                                               pot_3P1, spec_3P1, cont_3P1)

### Partial wave: 1D2

In [13]:
jmom, wave = 2, '1D2'
pot_1D2, spec_1D2, cont_1D2 = uncoupled_potential(jmom, lecs_1D2, 
                                                  spectral, contacts, wave)
V_1D2, V0_1D2, V1_1D2 = full_pot.no_interp_pot(jmom, wave, lecs_1D2, 
                                               pot_1D2, spec_1D2, cont_1D2)

### Partial wave: 3D2

In [14]:
jmom, wave = 2, '3D2'
pot_3D2, spec_3D2, cont_3D2 = uncoupled_potential(jmom, lecs_3D2, 
                                                  spectral, contacts, wave)
V_3D2, V0_3D2, V1_3D2 = full_pot.no_interp_pot(jmom, wave, lecs_3D2, 
                                               pot_3D2, spec_3D2, cont_3D2)

### Partial wave: 1F3

In [15]:
jmom, wave = 3, '1F3'
pot_1F3, spec_1F3, cont_1F3 = uncoupled_potential(jmom, lecs_1F3, 
                                                  spectral, contacts, wave)
V_1F3, V0_1F3, V1_1F3 = full_pot.no_interp_pot(jmom, wave, lecs_1F3, 
                                               pot_1F3, spec_1F3, cont_1F3)

### Partial wave: 3F3

In [16]:
jmom, wave = 3, '3F3'
pot_3F3, spec_3F3, cont_3F3 = uncoupled_potential(jmom, lecs_3F3, 
                                                  spectral, contacts, wave)
V_3F3, V0_3F3, V1_3F3 = full_pot.no_interp_pot(jmom, wave, lecs_3F3, 
                                               pot_3F3, spec_3F3, cont_3F3)

## Coupled partial waves

### Partial wave: 3S1/3D1

In [17]:
jmom, wave = 1, '3S1/3D1'
pot_3S1_3D1, spec_3S1_3D1, cont_3S1_3D1 = coupled_potential(jmom, lecs_3S1_3D1, 
                                                            spectral, contacts, wave)
V_3S1_3D1, V0_3S1_3D1, V1_3S1_3D1 = full_pot.no_interp_pot(jmom, wave, 
                                                           lecs_3S1_3D1, pot_3S1_3D1, 
                                                           spec_3S1_3D1, cont_3S1_3D1)

### Partial wave: 3P2/3F2

In [18]:
jmom, wave = 2, '3P2/3F2'
pot_3P2_3F2, spec_3P2_3F2, cont_3P2_3F2 = coupled_potential(jmom, lecs_3P2_3F2, 
                                                            spectral, contacts, wave)
V_3P2_3F2, V0_3P2_3F2, V1_3P2_3F2 = full_pot.no_interp_pot(jmom, wave, 
                                                           lecs_3P2_3F2, pot_3P2_3F2, 
                                                           spec_3P2_3F2, cont_3P2_3F2)

### Partial wave: 3D3/3G3

In [19]:
jmom, wave = 3, '3D3/3G3'
pot_3D3_3G3, spec_3D3_3G3, cont_3D3_3G3 = coupled_potential(jmom, lecs_3D3_3G3, 
                                                            spectral, contacts, wave)
V_3D3_3G3, V0_3D3_3G3, V1_3D3_3G3 = full_pot.no_interp_pot(jmom, wave, 
                                                           lecs_3D3_3G3, pot_3D3_3G3, 
                                                           spec_3D3_3G3, cont_3D3_3G3)

### Partial wave: 3F4/3H4

In [20]:
jmom, wave = 4, '3F4/3H4'
pot_3F4_3H4, spec_3F4_3H4, cont_3F4_3H4 = coupled_potential(jmom, lecs_3F4_3H4, 
                                                            spectral, contacts, wave)
V_3F4_3H4, V0_3F4_3H4, V1_3F4_3H4 = full_pot.no_interp_pot(jmom, wave, 
                                                           lecs_3F4_3H4, pot_3F4_3H4, 
                                                           spec_3F4_3H4, cont_3F4_3H4)

### Partial wave: j = 4 no contact dependence

In [21]:
jmom = 4
V_1 = full_pot.uncoupled_potential_no_contacts_interp(spectral[jmom][0])
V_2 = full_pot.uncoupled_potential_no_contacts_interp(spectral[jmom][1])

### Partial wave: j > 4 no contact dependence

In [22]:
V_no_contacts = [
    [
     full_pot.uncoupled_potential_no_contacts_interp(spectral[j + 5][idx]),
    ] 
    for j in range(jmax - 4) for idx in range(len(spectral[j + 5]))
]

## Save non-interpolated potential to h5 file

In [23]:
file_name = '../../../data/SMS_n4lo+_' + 'Lambda' + str(value) + 'MeV_jmax-' \
            + str(jmax) + '_' + interaction + '_' + str(chiral_order) \
            + '_' + str(ki) + str(cut_reg1) + str(cut_reg2) \
            + '_' + str(pts_reg1) + str(pts_reg2) + str(pts_reg3) + '.h5'
hf_sms = h5py.File(file_name, 'w')

hf_sms.create_dataset('jmax', data=jmax)
hf_sms.create_dataset('interaction', data=interaction)
hf_sms.create_dataset('chiral_order', data=chiral_order)
hf_sms.create_dataset('cutoff', data=cutoff)

hf_sms.create_dataset('E', data=E_MeV)
hf_sms.create_dataset('k', data=k_cm)
hf_sms.create_dataset('degrees', data=degrees)
hf_sms.create_dataset('ps', data=nodes)
hf_sms.create_dataset('ws', data=weights)
hf_sms.create_dataset('mesh_nodes', data=mesh_nodes)
hf_sms.create_dataset('pts_per_region', data=pts_per_region)

hf_sms.create_dataset('V_1S0', data=V_1S0)
hf_sms.create_dataset('V0_1S0', data=V0_1S0)
hf_sms.create_dataset('V1_1S0', data=V1_1S0)

hf_sms.create_dataset('V_3P0', data=V_3P0)
hf_sms.create_dataset('V0_3P0', data=V0_3P0)
hf_sms.create_dataset('V1_3P0', data=V1_3P0)

hf_sms.create_dataset('V_1P1', data=V_1P1)
hf_sms.create_dataset('V0_1P1', data=V0_1P1)
hf_sms.create_dataset('V1_1P1', data=V1_1P1)

hf_sms.create_dataset('V_3P1', data=V_3P1)
hf_sms.create_dataset('V0_3P1', data=V0_3P1)
hf_sms.create_dataset('V1_3P1', data=V1_3P1)

hf_sms.create_dataset('V_1D2', data=V_1D2)
hf_sms.create_dataset('V0_1D2', data=V0_1D2)
hf_sms.create_dataset('V1_1D2', data=V1_1D2)

hf_sms.create_dataset('V_3D2', data=V_3D2)
hf_sms.create_dataset('V0_3D2', data=V0_3D2)
hf_sms.create_dataset('V1_3D2', data=V1_3D2)

hf_sms.create_dataset('V_1F3', data=V_1F3)
hf_sms.create_dataset('V0_1F3', data=V0_1F3)
hf_sms.create_dataset('V1_1F3', data=V1_1F3)

hf_sms.create_dataset('V_3F3', data=V_3F3)
hf_sms.create_dataset('V0_3F3', data=V0_3F3)
hf_sms.create_dataset('V1_3F3', data=V1_3F3)

hf_sms.create_dataset('V_1', data=V_1)
hf_sms.create_dataset('V_2', data=V_2)

hf_sms.create_dataset('V_3S1_3D1', data=V_3S1_3D1)
hf_sms.create_dataset('V0_3S1_3D1', data=V0_3S1_3D1)
hf_sms.create_dataset('V1_3S1_3D1', data=V1_3S1_3D1)

hf_sms.create_dataset('V_3P2_3F2', data=V_3P2_3F2)
hf_sms.create_dataset('V0_3P2_3F2', data=V0_3P2_3F2)
hf_sms.create_dataset('V1_3P2_3F2', data=V1_3P2_3F2)

hf_sms.create_dataset('V_3D3_3G3', data=V_3D3_3G3)
hf_sms.create_dataset('V0_3D3_3G3', data=V0_3D3_3G3)
hf_sms.create_dataset('V1_3D3_3G3', data=V1_3D3_3G3)

hf_sms.create_dataset('V_3F4_3H4', data=V_3F4_3H4)
hf_sms.create_dataset('V0_3F4_3H4', data=V0_3F4_3H4)
hf_sms.create_dataset('V1_3F4_3H4', data=V1_3F4_3H4)

hf_sms.create_dataset('V_no_contacts', data=V_no_contacts)

hf_sms.close()